## Определение перспективного тарифа телекомпании

### Описание проекта

У компании есть два тарифных планах "Смарт" и "Ультра", необходимо определить какой тариф приносит больше денег по выборке 500 клиентов.
  
| |«Смарт»|«Ультра»|
|--|--|--|
|Ежемесячная оплата|550|1950|
|Включено:|||
|Минут разговора|500|3000|
|Сообщений|50|1000|
|Гб трафика|15|30|
|Стоимость свыше:|||
|Минут разговора|3|1|
|Сообщений|3|1|
|Гб трафика|200|150|

Проверить 2 гипотезы:  
- Средняя выручка тарифов «Ультра» и «Смарт» различаются 
- Средняя выручка пользователей из Москвы отличается от выручки пользователей из других регионов

### Знакомство с данными и придение к нужным типам данных

In [56]:
import pandas as pd
import math as mt
import numpy as np

In [57]:
try:
    df_users = pd.read_csv('users.csv', parse_dates=['churn_date', 'reg_date'])
    df_calls = pd.read_csv('calls.csv', parse_dates=['call_date'])
    df_messages = pd.read_csv('messages.csv', parse_dates=['message_date'])
    df_internet = pd.read_csv('internet.csv', parse_dates=['session_date'])
    df_tariffs = pd.read_csv('tariffs.csv')
except:
    df_users = pd.read_csv('/datasets/users.csv', parse_dates=['churn_date', 'reg_date'])
    df_calls = pd.read_csv('/datasets/calls.csv', parse_dates=['call_date'])
    df_messages = pd.read_csv('/datasets/messages.csv', parse_dates=['message_date'])
    df_internet = pd.read_csv('/datasets/internet.csv', parse_dates=['session_date'])
    df_tariffs = pd.read_csv('/datasets/tariffs.csv')

In [58]:
df_users.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 500 entries, 0 to 499
Data columns (total 8 columns):
 #   Column      Non-Null Count  Dtype         
---  ------      --------------  -----         
 0   user_id     500 non-null    int64         
 1   age         500 non-null    int64         
 2   churn_date  38 non-null     datetime64[ns]
 3   city        500 non-null    object        
 4   first_name  500 non-null    object        
 5   last_name   500 non-null    object        
 6   reg_date    500 non-null    datetime64[ns]
 7   tariff      500 non-null    object        
dtypes: datetime64[ns](2), int64(2), object(4)
memory usage: 31.4+ KB


In [59]:
df_users[df_users.duplicated(subset=['last_name', 'first_name'])]

,user_id,age,churn_date,city,first_name,last_name,reg_date,tariff


In [60]:
df_users['tariff'].value_counts()

smart    351
ultra    149
Name: tariff, dtype: int64

`df_users` - дубликатов нет, пропущенные значения только в дате прекращения пользования тарифом(но согласно документации, данный тариф действующий). Все данные в правильных форматах.

In [61]:
df_calls.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 202607 entries, 0 to 202606
Data columns (total 4 columns):
 #   Column     Non-Null Count   Dtype         
---  ------     --------------   -----         
 0   id         202607 non-null  object        
 1   call_date  202607 non-null  datetime64[ns]
 2   duration   202607 non-null  float64       
 3   user_id    202607 non-null  int64         
dtypes: datetime64[ns](1), float64(1), int64(1), object(1)
memory usage: 6.2+ MB


In [62]:
df_calls[df_calls.duplicated(subset='id')]

,id,call_date,duration,user_id


`df_calls` - необходимо поправить duration(длительность звонка в минутах), т.к. в описании тарифа должно быть округление до целого. соответственно привести к int. еще конечно в идеале id хранить в int, но не уверен что стоит сейчас это делать.

In [63]:
df_calls.head(5)

,id,call_date,duration,user_id
0,1000_0,2018-07-25,0.00,1000
1,1000_1,2018-08-17,0.00,1000
2,1000_2,2018-06-11,2.85,1000
3,1000_3,2018-09-21,13.80,1000
4,1000_4,2018-12-15,5.18,1000


In [64]:
df_calls['duration'] = np.ceil(df_calls['duration']).astype(int)
df_calls.head(5)

,id,call_date,duration,user_id
0,1000_0,2018-07-25,0,1000
1,1000_1,2018-08-17,0,1000
2,1000_2,2018-06-11,3,1000
3,1000_3,2018-09-21,14,1000
4,1000_4,2018-12-15,6,1000


In [65]:
df_calls.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 202607 entries, 0 to 202606
Data columns (total 4 columns):
 #   Column     Non-Null Count   Dtype         
---  ------     --------------   -----         
 0   id         202607 non-null  object        
 1   call_date  202607 non-null  datetime64[ns]
 2   duration   202607 non-null  int32         
 3   user_id    202607 non-null  int64         
dtypes: datetime64[ns](1), int32(1), int64(1), object(1)
memory usage: 5.4+ MB


In [66]:
df_messages.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 123036 entries, 0 to 123035
Data columns (total 3 columns):
 #   Column        Non-Null Count   Dtype         
---  ------        --------------   -----         
 0   id            123036 non-null  object        
 1   message_date  123036 non-null  datetime64[ns]
 2   user_id       123036 non-null  int64         
dtypes: datetime64[ns](1), int64(1), object(1)
memory usage: 2.8+ MB


`df_messages` - Проблем не обнаружено, вроде все ок и все в правильном типе. Разве что можно поменять id на int

In [67]:
df_internet.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 149396 entries, 0 to 149395
Data columns (total 5 columns):
 #   Column        Non-Null Count   Dtype         
---  ------        --------------   -----         
 0   Unnamed: 0    149396 non-null  int64         
 1   id            149396 non-null  object        
 2   mb_used       149396 non-null  float64       
 3   session_date  149396 non-null  datetime64[ns]
 4   user_id       149396 non-null  int64         
dtypes: datetime64[ns](1), float64(1), int64(2), object(1)
memory usage: 5.7+ MB


In [68]:
df_internet.head(5)

,Unnamed: 0,id,mb_used,session_date,user_id
0,0,1000_0,112.95,2018-11-25,1000
1,1,1000_1,1052.81,2018-09-07,1000
2,2,1000_2,1197.26,2018-06-25,1000
3,3,1000_3,550.27,2018-08-22,1000
4,4,1000_4,302.56,2018-09-24,1000


`df_internet` : Затисалась странная колонка, которую стоит удалить. мб надо будет перевести в гб, после того сделается сводная таблица

In [69]:
df_internet.drop(columns='Unnamed: 0', inplace=True)

In [70]:
df_tariffs.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2 entries, 0 to 1
Data columns (total 8 columns):
 #   Column                 Non-Null Count  Dtype 
---  ------                 --------------  ----- 
 0   messages_included      2 non-null      int64 
 1   mb_per_month_included  2 non-null      int64 
 2   minutes_included       2 non-null      int64 
 3   rub_monthly_fee        2 non-null      int64 
 4   rub_per_gb             2 non-null      int64 
 5   rub_per_message        2 non-null      int64 
 6   rub_per_minute         2 non-null      int64 
 7   tariff_name            2 non-null      object
dtypes: int64(7), object(1)
memory usage: 256.0+ bytes


In [71]:
df_tariffs.head(5)

,messages_included,mb_per_month_included,minutes_included,rub_monthly_fee,rub_per_gb,rub_per_message,rub_per_minute,tariff_name
0,50,15360,500,550,200,3,3,smart
1,1000,30720,3000,1950,150,1,1,ultra


`df_tariffs` - все ок.

**Общий вывод**:  
В целом данные хорошие, пропусков нет, дубликатов нет, возможно появяться скрытые проблемы после того как начну группировать/ проводить аналитику

### Добавление колонок и создание сводных таблиц

In [83]:
# Проверка что все действия за 2018год и не вписалось ничего лишнего
df_calls['call_date'].describe()
df_internet['session_date'].describe()
df_messages['message_date'].describe()

C:\Users\Bacek\AppData\Local\Temp/ipykernel_17228/3848770814.py:1: FutureWarning: Treating datetime data as categorical rather than numeric in `.describe` is deprecated and will be removed in a future version of pandas. Specify `datetime_is_numeric=True` to silence this warning and adopt the future behavior now.
  df_calls['call_date'].describe()
C:\Users\Bacek\AppData\Local\Temp/ipykernel_17228/3848770814.py:2: FutureWarning: Treating datetime data as categorical rather than numeric in `.describe` is deprecated and will be removed in a future version of pandas. Specify `datetime_is_numeric=True` to silence this warning and adopt the future behavior now.
  df_internet['session_date'].describe()
C:\Users\Bacek\AppData\Local\Temp/ipykernel_17228/3848770814.py:3: FutureWarning: Treating datetime data as categorical rather than numeric in `.describe` is deprecated and will be removed in a future version of pandas. Specify `datetime_is_numeric=True` to silence this warning and adopt the fut

count                  123036
unique                    364
top       2018-12-31 00:00:00
freq                      859
first     2018-01-02 00:00:00
last      2018-12-31 00:00:00
Name: message_date, dtype: object

In [77]:
df_calls['call_month'] = df_calls['call_date'].dt.month
df_internet['internet_month'] = df_internet['session_date'].dt.month
df_messages['message_month'] = df_messages['message_date'].dt.month

In [101]:
df_internet.columns

Index(['id', 'mb_used', 'session_date', 'user_id', 'internet_month'], dtype='object')

In [121]:
df_pivot_calls = df_calls.pivot_table(
    values=['duration'], index=['user_id', 'call_month'], aggfunc=['sum']).reset_index()
df_pivot_calls.columns = ['user_id', 'month', 'sum_duration']

In [122]:
df_pivot_internet = df_internet.pivot_table(
    values=['mb_used'], index=['user_id','internet_month'], aggfunc=['sum']).reset_index()
df_pivot_internet.columns = ['user_id','month', 'mb_used']

Сразу переведем мегабайты в гигабайты, которые требуются для расчета стоитмости

In [123]:
df_pivot_internet['gb_used'] = np.ceil(df_pivot_internet['mb_used'] / 1024).astype(int)
df_pivot_internet.drop(columns=['mb_used'], inplace=True)

In [124]:
df_messages.columns

Index(['id', 'message_date', 'user_id', 'message_month'], dtype='object')

In [125]:
df_pivot_messages = df_messages.pivot_table(
    values=['id'], index=['user_id', 'message_month'], aggfunc=['count']
).reset_index()
df_pivot_messages.columns = ['user_id', 'month', 'count_message']

In [ ]:
df_users['reg_date'].describe()

C:\Users\Bacek\AppData\Local\Temp/ipykernel_17228/656204730.py:1: FutureWarning: Treating datetime data as categorical rather than numeric in `.describe` is deprecated and will be removed in a future version of pandas. Specify `datetime_is_numeric=True` to silence this warning and adopt the future behavior now.
  df_users['reg_date'].describe()


count                     500
unique                    270
top       2018-05-28 00:00:00
freq                        8
first     2018-01-01 00:00:00
last      2018-12-31 00:00:00
Name: reg_date, dtype: object

In [157]:
df_pivot_all = df_users.merge(df_pivot_calls, how='outer', on=['user_id'])

In [158]:
df_pivot_all.columns

Index(['user_id', 'age', 'churn_date', 'city', 'first_name', 'last_name',
       'reg_date', 'tariff', 'month', 'sum_duration'],
      dtype='object')

In [155]:
df_pivot_all_internet = df_users.merge(df_pivot_internet, how='outer',on='user_id')

In [159]:
df_pivot_all = df_pivot_all.merge(df_pivot_all_internet, how='outer', on=['user_id', 'age', 'churn_date', 'city', 'first_name', 'last_name',
       'reg_date', 'tariff', 'month'])

In [ ]:
df_